# Bonds

Copyright (&copy;) 2008 Florent Grenier
Copyright (&copy;) 2010 Lluis Pujol Bajador

This file is part of QuantLib, a free-software/open-source library
for financial quantitative analysts and developers - https://www.quantlib.org/

QuantLib is free software: you can redistribute it and/or modify it
under the terms of the QuantLib license.  You should have received a
copy of the license along with this program; if not, please email
<quantlib-dev@lists.sf.net>. The license is also available online at
<https://www.quantlib.org/license.shtml>.

This program is distributed in the hope that it will be useful, but WITHOUT
ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS
FOR A PARTICULAR PURPOSE.  See the license for more details.

   This example shows how to set up a term structure and then price
   some simple bonds. The last part is dedicated to peripherical
   computations such as "Yield to Price" or "Price to Yield"

In [1]:
import QuantLib as ql
import pandas as pd

In [2]:
interactive = 'get_ipython' in globals()

### Global data

In [3]:
calendar = ql.TARGET()
settlementDate = ql.Date(18, ql.September, 2008)
settlementDate = calendar.adjust(settlementDate)

In [4]:
fixingDays = 3
settlementDays = 3

In [5]:
todaysDate = calendar.advance(settlementDate, -fixingDays, ql.Days)
ql.Settings.instance().evaluationDate = todaysDate

In [6]:
print("Today: " + str(todaysDate))
print("Settlement Date: " + str(settlementDate))

Today: September 15th, 2008
Settlement Date: September 18th, 2008


### Market quotes

In [7]:
zcQuotes = [(0.0096, ql.Period(3, ql.Months)), (0.0145, ql.Period(6, ql.Months)), (0.0194, ql.Period(1, ql.Years))]

In [8]:
zcBondsDayCounter = ql.Actual365Fixed()

In [9]:
zcHelpers = [
    ql.DepositRateHelper(
        ql.QuoteHandle(ql.SimpleQuote(r)), tenor, fixingDays, calendar, ql.ModifiedFollowing, True, zcBondsDayCounter
    )
    for (r, tenor) in zcQuotes
]

### Setup bonds

In [10]:
redemption = 100.0
numberOfBonds = 5

In [11]:
bondQuotes = [
    (ql.Date(15, ql.March, 2005), ql.Date(31, ql.August, 2010), 0.02375, 100.390625),
    (ql.Date(15, ql.June, 2005), ql.Date(31, ql.August, 2011), 0.04625, 106.21875),
    (ql.Date(30, ql.June, 2006), ql.Date(31, ql.August, 2013), 0.03125, 100.59375),
    (ql.Date(15, ql.November, 2002), ql.Date(15, ql.August, 2018), 0.04000, 101.6875),
    (ql.Date(15, ql.May, 1987), ql.Date(15, ql.May, 2038), 0.04500, 102.140625),
]

### Definition of the rate helpers

In [12]:
bondsHelpers = []

In [13]:
for issueDate, maturity, couponRate, marketQuote in bondQuotes:
    schedule = ql.Schedule(
        issueDate,
        maturity,
        ql.Period(ql.Semiannual),
        ql.UnitedStates(ql.UnitedStates.GovernmentBond),
        ql.Unadjusted,
        ql.Unadjusted,
        ql.DateGeneration.Backward,
        False,
    )
    bondsHelpers.append(
        ql.FixedRateBondHelper(
            ql.QuoteHandle(ql.SimpleQuote(marketQuote)),
            settlementDays,
            100.0,
            schedule,
            [couponRate],
            ql.ActualActual(ql.ActualActual.Bond),
            ql.Unadjusted,
            redemption,
            issueDate,
        )
    )

###  Curve building

In [14]:
termStructureDayCounter = ql.ActualActual(ql.ActualActual.ISDA)

In [15]:
bondInstruments = zcHelpers + bondsHelpers

In [16]:
bondDiscountingTermStructure = ql.PiecewiseFlatForward(settlementDate, bondInstruments, termStructureDayCounter)

### Building of the LIBOR forecasting curve

In [17]:
dQuotes = [
    (0.043375, ql.Period(1, ql.Weeks)),
    (0.031875, ql.Period(1, ql.Months)),
    (0.0320375, ql.Period(3, ql.Months)),
    (0.03385, ql.Period(6, ql.Months)),
    (0.0338125, ql.Period(9, ql.Months)),
    (0.0335125, ql.Period(1, ql.Years)),
]
sQuotes = [
    (0.0295, ql.Period(2, ql.Years)),
    (0.0323, ql.Period(3, ql.Years)),
    (0.0359, ql.Period(5, ql.Years)),
    (0.0412, ql.Period(10, ql.Years)),
    (0.0433, ql.Period(15, ql.Years)),
]

In [18]:
depositDayCounter = ql.Actual360()
depositHelpers = [
    ql.DepositRateHelper(
        ql.QuoteHandle(ql.SimpleQuote(rate)), tenor, fixingDays, calendar, ql.ModifiedFollowing, True, depositDayCounter
    )
    for rate, tenor in dQuotes
]

In [19]:
swFixedLegFrequency = ql.Annual
swFixedLegConvention = ql.Unadjusted
swFixedLegDayCounter = ql.Thirty360(ql.Thirty360.European)
swFloatingLegIndex = ql.Euribor6M()
forwardStart = ql.Period(1, ql.Days)
swapHelpers = [
    ql.SwapRateHelper(
        ql.QuoteHandle(ql.SimpleQuote(rate)),
        tenor,
        calendar,
        swFixedLegFrequency,
        swFixedLegConvention,
        swFixedLegDayCounter,
        swFloatingLegIndex,
        ql.QuoteHandle(),
        forwardStart,
    )
    for rate, tenor in sQuotes
]

In [20]:
depoSwapInstruments = depositHelpers + swapHelpers

In [21]:
depoSwapTermStructure = ql.PiecewiseFlatForward(settlementDate, depoSwapInstruments, termStructureDayCounter)

### Pricing

Term structures that will be used for pricing:
the one used for discounting cash flows...

In [22]:
discountingTermStructure = ql.RelinkableYieldTermStructureHandle()

...and the one used for forward rate forecasting.

In [23]:
forecastingTermStructure = ql.RelinkableYieldTermStructureHandle()

Bonds to be priced:

In [24]:
faceAmount = 100

In [25]:
bondEngine = ql.DiscountingBondEngine(discountingTermStructure)

a zero coupon bond...

In [26]:
zeroCouponBond = ql.ZeroCouponBond(
    settlementDays,
    ql.UnitedStates(ql.UnitedStates.GovernmentBond),
    faceAmount,
    ql.Date(15, ql.August, 2013),
    ql.Following,
    116.92,
    ql.Date(15, ql.August, 2003),
)

In [27]:
zeroCouponBond.setPricingEngine(bondEngine)

...a fixed 4.5% US Treasury note...

In [28]:
fixedBondSchedule = ql.Schedule(
    ql.Date(15, ql.May, 2007),
    ql.Date(15, ql.May, 2017),
    ql.Period(ql.Semiannual),
    ql.UnitedStates(ql.UnitedStates.GovernmentBond),
    ql.Unadjusted,
    ql.Unadjusted,
    ql.DateGeneration.Backward,
    False,
)

In [29]:
fixedRateBond = ql.FixedRateBond(
    settlementDays,
    faceAmount,
    fixedBondSchedule,
    [0.045],
    ql.ActualActual(ql.ActualActual.Bond),
    ql.ModifiedFollowing,
    100.0,
    ql.Date(15, ql.May, 2007),
)

In [30]:
fixedRateBond.setPricingEngine(bondEngine)

...and a floating rate bond paying 3M USD Libor + 0.1%
(should and will be priced on another curve later).

In [31]:
liborTermStructure = ql.RelinkableYieldTermStructureHandle()

In [32]:
libor3m = ql.USDLibor(ql.Period(3, ql.Months), liborTermStructure)
libor3m.addFixing(ql.Date(17, ql.April, 2008), 0.028175)
libor3m.addFixing(ql.Date(17, ql.July, 2008), 0.0278625)

In [33]:
floatingBondSchedule = ql.Schedule(
    ql.Date(21, ql.October, 2005),
    ql.Date(21, ql.October, 2010),
    ql.Period(ql.Quarterly),
    ql.UnitedStates(ql.UnitedStates.NYSE),
    ql.Unadjusted,
    ql.Unadjusted,
    ql.DateGeneration.Backward,
    True,
)

In [34]:
floatingRateBond = ql.FloatingRateBond(
    settlementDays,
    faceAmount,
    floatingBondSchedule,
    libor3m,
    ql.Actual360(),
    ql.ModifiedFollowing,
    spreads=[0.001],
    issueDate=ql.Date(21, ql.October, 2005),
)

In [35]:
floatingRateBond.setPricingEngine(bondEngine)

In [36]:
forecastingTermStructure.linkTo(depoSwapTermStructure)
discountingTermStructure.linkTo(bondDiscountingTermStructure)

In [37]:
liborTermStructure.linkTo(depoSwapTermStructure)

In [38]:
data = []
data.append(
    (zeroCouponBond.cleanPrice(), fixedRateBond.cleanPrice(), floatingRateBond.cleanPrice())
)
data.append(
    (zeroCouponBond.dirtyPrice(), fixedRateBond.dirtyPrice(), floatingRateBond.dirtyPrice())
)
data.append(
    (zeroCouponBond.accruedAmount(),
     fixedRateBond.accruedAmount(),
     floatingRateBond.accruedAmount())
)
data.append(
    (None, fixedRateBond.previousCouponRate(), floatingRateBond.previousCouponRate())
)
data.append(
    (None, fixedRateBond.nextCouponRate(), floatingRateBond.nextCouponRate())
)
data.append(
    (zeroCouponBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual),
     fixedRateBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual),
     floatingRateBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual))
)

df = pd.DataFrame(data, columns=["ZC", "Fixed", "Floating"],
                  index=["Clean price", "Dirty price", "Accrued coupon",
                         "Previous coupon rate", "Next coupon rate", "Yield"])
if not interactive:
    print(df)
df

,ZC,Fixed,Floating
Clean price,100.922178,106.127528,101.655903
Dirty price,100.922178,107.668289,102.128927
Accrued coupon,0.000000,1.540761,0.473024
Previous coupon rate,NaN,0.045000,0.029175
Next coupon rate,NaN,0.045000,0.028862
Yield,0.030001,0.036476,0.022007


A few other computations:

Yield to clean price:

In [39]:
floatingRateBond.cleanPrice(
    floatingRateBond.bondYield(ql.Actual360(), ql.Compounded, ql.Annual),
    ql.Actual360(),
    ql.Compounded,
    ql.Annual,
    settlementDate,
)

101.65590321313849

Clean price to yield:

In [40]:
floatingRateBond.bondYield(
    floatingRateBond.cleanPrice(),
    ql.Actual360(),
    ql.Compounded,
    ql.Annual,
    settlementDate
)

0.022007488506948527